In [1]:
print("Importing libraries ... ")
import numpy as np
import matplotlib.pyplot as plt
import os
import json
import Puissance_4_bitboard
print("Done")

Importing libraries ... 
Done


In [2]:
file_dir = os.getcwd()+"\\"
file_data = "data_set_Puissance_4_20.json"

In [3]:
def load_json(json_file):
    with open(json_file) as f:
        data = json.load(f)
        return data

In [6]:
data = load_json(file_dir+file_data)
(X_train, Y_train),\
(X_val, Y_val),\
(X_test, Y_test) = (data["train"]["input"],data["train"]["output"]),\
                   (data["validation"]["input"],data["validation"]["output"]),\
                   (data["test"]["input"],data["test"]["output"])

In [7]:
(X_train, Y_train),\
(X_val, Y_val),\
(X_test, Y_test) = (X_train[:10000], Y_train[:10000]), (X_val[:10000], Y_val[:10000]), (X_test[:10000], Y_test[:10000])

In [8]:
import tensorflow.keras
from tensorflow.keras import models, layers
from tensorflow.keras import optimizers
from sklearn.neural_network import MLPRegressor
import tensorflow.keras.optimizers as optimizers
import optuna
import pickle

In [9]:
def r2_score(y_true, y_pred):
    return 1 - (np.sum((y_true - y_pred)**2) / np.sum((y_true - y_true.mean())**2))

def nrmse(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    return np.sqrt((np.sum(y_true - y_pred)**2) / len(y_true))

In [10]:
new_X_train = [[[[(int(val)&(1<<(i*6+j)))==(1<<(i*6+j)) for val in X_train[k].split()] for j in range(7)] for i in range(6)] for k in range(len(X_train))]
new_X_val = [[[[(int(val)&(1<<(i*6+j)))==(1<<(i*6+j)) for val in X_val[k].split()] for j in range(7)] for i in range(6)] for k in range(len(X_val))]
new_X_test = [[[[(int(val)&(1<<(i*6+j)))==(1<<(i*6+j)) for val in X_test[k].split()] for j in range(7)] for i in range(6)] for k in range(len(X_test))]

In [11]:
print(np.array(X_train).shape)

(10000,)


In [12]:
new_Y_train = Y_train
new_Y_val = Y_val
new_Y_test = Y_test

In [13]:
new_X_train += [[new_X_train[i][j][::-1] for j in range(len(new_X_train[i]))] for i in range(len(new_X_train))]
new_Y_train += new_Y_train

new_X_val += [[new_X_val[i][j][::-1] for j in range(len(new_X_val[i]))] for i in range(len(new_X_val))]
new_Y_val += new_Y_val

new_X_test += [[new_X_test[i][j][::-1] for j in range(len(new_X_test[i]))] for i in range(len(new_X_test))]
new_Y_test += new_Y_test

In [14]:
print(np.array(new_X_train).shape)

(20000, 6, 7, 2)


In [15]:
STUDY_NAME = "10k_50_trials"
STR_PATH = "sqlite:///"+STUDY_NAME+"/"+STUDY_NAME+".db"
loaded_study = optuna.load_study(study_name=STUDY_NAME, storage=STR_PATH)
trial = loaded_study.best_trial
print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

OperationalError: (sqlite3.OperationalError) unable to open database file
(Background on this error at: http://sqlalche.me/e/14/e3q8)

In [16]:
new_train_2 = np.squeeze(np.array(new_X_train).reshape([len(new_X_train),-1,1]))
new_val_2 = np.squeeze(np.array(new_X_val).reshape([len(new_X_val),-1,1]))
new_test_2 = np.squeeze(np.array(new_X_test).reshape([len(new_X_test),-1,1]))

In [17]:
print(len(new_train_2))
print(len(new_X_train))
print(new_train_2[0])

20000
20000
[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False  True False  True False  True False  True False
 False False False False False  True False False  True False False  True]


In [19]:
regr = MLPRegressor(random_state=1,\
                solver = 'sgd',\
                hidden_layer_sizes=(114,57,89),\
                activation = 'tanh',\
                max_iter=86352764,\
                alpha=0.5602672200939348,\
                max_fun = 1000).fit(new_train_2, new_Y_train)

In [20]:
regr.score(new_val_2,new_Y_val)

0.011240351837250862

In [22]:
print(regr.predict(new_val_2[:10]))
print(new_Y_val[:10])

[[0.43561202 0.56173502]
 [0.43487    0.56082433]
 [0.44073978 0.55521967]
 [0.44960442 0.54637289]
 [0.49276962 0.50390688]
 [0.46839228 0.52676044]
 [0.48193586 0.51439477]
 [0.50169427 0.49272417]
 [0.46694818 0.52733314]
 [0.43314907 0.56343172]]
[[0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0]]
